In [9]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install wordcloud
!pip install nltk
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 3.5 MB/s eta 0:00:00


## Understanding the data

In [43]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import re

In [44]:
df = pd.read_csv('dataset.csv')

In [45]:
df.head()

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


In [46]:
df.shape

(200000, 2)

Check for duplicates and missing values

In [47]:
df[df.duplicated()]

,text,humor


In [48]:
df[df.isna().any(axis=1)]

,text,humor


We have no duplicates and no missing values.

In [49]:
df['humor'].value_counts()

humor
False    100000
True     100000
Name: count, dtype: int64

We can see that we have a balanced dataset, so we do not have to use oversampling techniques.

## Further preparation of data for EDA

In [50]:
import re

import contractions
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from numpy import *
import seaborn as sns
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS


# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Counts and Length:
1. Character count
2. Word count
3. Mean word length
4. Mean sentence length

In [51]:
# length of a sample piece of text
length = len(df['text'][0])
print(f'Length of a sample review: {length}')

Length of a sample review: 53


In [52]:
df['length'] = df['text'].str.len()
df.head()

,text,humor,length
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False,53
1,Watch: darvish gave hitter whiplash with slow ...,False,51
2,What do you call a turtle without its shell? d...,True,50
3,5 reasons the 2016 election feels so personal,False,45
4,"Pasco police shot mexican migrant from behind,...",False,64


Getting the word counts in each piece of text

In [53]:
def word_count(text):
    text_list = text.split()
    return len(text_list)

In [54]:
df['word_count'] = df['text'].apply(word_count)
df.head()

,text,humor,length,word_count
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False,53,10
1,Watch: darvish gave hitter whiplash with slow ...,False,51,8
2,What do you call a turtle without its shell? d...,True,50,10
3,5 reasons the 2016 election feels so personal,False,45,8
4,"Pasco police shot mexican migrant from behind,...",False,64,10


Mean word length: The average length of words

In [55]:
df['mean_word_length'] = df['text'].map(lambda t: np.mean([len(word) for word in t.split()]))
df.head()

,text,humor,length,word_count,mean_word_length
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False,53,10,4.40
1,Watch: darvish gave hitter whiplash with slow ...,False,51,8,5.50
2,What do you call a turtle without its shell? d...,True,50,10,4.10
3,5 reasons the 2016 election feels so personal,False,45,8,4.75
4,"Pasco police shot mexican migrant from behind,...",False,64,10,5.50


continue from: https://www.kaggle.com/code/harshsingh2209/complete-guide-to-eda-on-text-data

Removal of punctuation marks